In [1]:
import pandas as pd
import numpy as np
import gzip
from collections import defaultdict
import matplotlib.pyplot as plt
import pickle
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>");

In [2]:
curr_dir = !pwd
pfam_version = "32"
filename = "Pfam-A.hmm.gz"
pfam_file = gzip.open(curr_dir[0]+"/v"+pfam_version+"/"+filename,'r')

In [3]:
#Constants
EOF = "//"

In [4]:
#%%time 
#Reading the -log(prob) of the match states emissions from the Pfam HMM file

domains_hmm_dict = {}
print_flag = True

while (print_flag):
    print_flag = False
    hmm_log_prob_dict = defaultdict(list)
    for line in pfam_file:
        print_flag = True
        if (line[0:4] == "NAME"):
            domain_name = line[6:-1]
        if (line[0:4] == "HMM "):
            aa = line.split()
            aa.remove("HMM")
            break

    states_cnt = 1
    for line in pfam_file:
        print_flag = True
        line_list = line.split()
        if (line_list[0] == EOF):
            break
        if (line_list[0] == str(states_cnt)):
            #Saving the probalities as a list for the corresponding HMM state
            prob_strs = line.split()[1:1+len(aa)]
            for i in prob_strs:
                if (i == "*"):
                    hmm_log_prob_dict[states_cnt].append(float('inf'))
                else:
                    hmm_log_prob_dict[states_cnt].append(float(i))
            states_cnt += 1

    #Checking if its the end of the entire file
    if (len(hmm_log_prob_dict.keys()) == 0):
        break
    domains_hmm_dict[domain_name] = hmm_log_prob_dict

In [5]:
#Saving to file
with open(curr_dir[0]+"/v"+pfam_version+"/domains_hmm_dict.pik", 'wb') as handle:
    pickle.dump(domains_hmm_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
%%time
#Converting to probalities, and saving a vector for each HMM match state
domains_hmm_prob_dict = {}
for domain_name in domains_hmm_dict.keys():
    hmm_prob_dict = {}
    for state in domains_hmm_dict[domain_name].keys():
        hmm_prob_dict[state] = 1/np.exp(domains_hmm_dict[domain_name][state])
    domains_hmm_prob_dict[domain_name] = hmm_prob_dict

#Saving to file
with open(curr_dir[0]+"/v"+pfam_version+"/domains_hmm_prob_dict.pik", 'wb') as handle:
    pickle.dump(domains_hmm_prob_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

CPU times: user 2min 14s, sys: 8.98 s, total: 2min 23s
Wall time: 2min 22s


In [4]:
#Reading the dictionary of HMM probabilities
with open(curr_dir[0]+"/../2.parse_Pfam/v"+pfam_version+"/domains_hmm_prob_dict.pik", 'rb') as handle:
    domains_hmm_prob_dict = pickle.load(handle)

### Looking at the distribution of emission prob.

In [5]:
emission_prob = []
for domain in domains_hmm_prob_dict.keys():
    for state in domains_hmm_prob_dict[domain]:
        emission_prob.extend(domains_hmm_prob_dict[domain][state])

In [10]:
len(emission_prob)

57215700

In [6]:
my_bins = [0,0.8,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,1]
hist_info = plt.hist(emission_prob, bins=my_bins)
plt.show()

In [15]:
plt.show()

In [8]:
hist_info[0]

array([  5.71942710e+07,   1.55690000e+04,   1.04300000e+03,
         1.01800000e+03,   8.75000000e+02,   7.38000000e+02,
         6.74000000e+02,   6.31000000e+02,   5.89000000e+02,
         4.40000000e+02,   3.57000000e+02,   3.95000000e+02])

In [7]:
hist_info[1]

array([ 0.  ,  0.8 ,  0.9 ,  0.91,  0.92,  0.93,  0.94,  0.95,  0.96,
        0.97,  0.98,  0.99,  1.  ])